#Урок 2. Парсинг HTML. Библиотека Beautiful soup.

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:


1.   Наименование вакансии.
2.   Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
3.   Ссылку на саму вакансию.
4.   Сайт, откуда собрана вакансия.



> *По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.*

In [4]:
import re
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd

In [5]:
# ссылка (это я для себя ее так перенесла, показалось удобнее :))
# https://hh.ru/search/vacancy?
# clusters=true&
# area=1&
# ored_clusters=true&
# enable_snippets=true&
# salary=&
# text=design&
# page=3

In [6]:
field_search = 'design'
last_page = 10
page = 0

params = {'text': field_search,
          'page': page}

url = 'https://hh.ru/vacancy/'
base_url = 'https://hh.ru/search/vacancy?clusters=true&area=1&ored_clusters=true&enable_snippets=true&salary=&'

headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'}

In [9]:
for page in range(last_page):
    params['page'] = page
    response = requests.get(base_url, params=params, headers=headers)
    dom = bs(response.text, 'html.parser')
    page = +1

In [19]:
vacancies = dom.find_all('div', {'class': 'vacancy-serp-item-body'})

vacancies_list = []
for vacancy in vacancies:
    vacancy_data = {} # на каждую итерацию будем содавать словарь с данными
    vacancy_name = vacancy.find('span', {'class': 'g-user-content'}).getText()
    vacancy_link = vacancy.find('a', {'class': 'bloko-link'})['href']
    vacancy_company = vacancy.find('div', {'class': 'vacancy-serp-item__meta-info-company'}).getText().replace(u'\xa0', u' ')
    vacancy_city = vacancy.find('div', {'class': 'bloko-text bloko-text_no-top-indent'}).getText()
    vacancy_salary = vacancy.find('span', {'class': 'bloko-header-section-3'})
    # salary
    if vacancy_salary:
        vacancy_salary = vacancy_salary.getText().replace('\xa0', ' ', 3)
        vacancy_salary = re.split(r'\s|-', vacancy_salary)
        if vacancy_salary[0] == 'до':
            vacancy_salary_min = None
            vacancy_salary_max = int(vacancy_salary[1]+vacancy_salary[2])
            vacancy_salary_currency = vacancy_salary[3]
        elif vacancy_salary[0] == 'от':
            vacancy_salary_min = int(vacancy_salary[1]+vacancy_salary[2])
            vacancy_salary_max = None
            vacancy_salary_currency = vacancy_salary[3]
        else:
            vacancy_salary_min = int(vacancy_salary[0] + vacancy_salary[1])
            vacancy_salary_max = int(vacancy_salary[3] + vacancy_salary[4])
            vacancy_salary_currency = vacancy_salary[5]


        #print(vacancy_salary[1]+vacancy_salary[2])
    else:
        vacancy_salary = None
        vacancy_salary_min = None
        vacancy_salary_max = None
        vacancy_salary_currency = None
        
    vacancy_data['name'] = vacancy_name
    vacancy_data['link'] = vacancy_link
    vacancy_data['company'] = vacancy_company
    vacancy_data['city'] = vacancy_city
    vacancy_data['salary_min'] = vacancy_salary_min
    vacancy_data['salary_max'] = vacancy_salary_max
    vacancy_data['salary_currency'] = vacancy_salary_currency
    vacancies_list.append(vacancy_data)


In [20]:
df = pd.DataFrame(vacancies_list)
print(df)

[]


In [16]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/file.csv')

AttributeError: ignored